## 安裝 Spark

In [0]:
! apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [0]:
! wget -q https://www-us.apache.org/dist/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz

In [0]:
! tar xf spark-2.4.5-bin-hadoop2.7.tgz

In [0]:
! pip install -q findspark

In [0]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-1.8.0-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.5-bin-hadoop2.7"

In [0]:

import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [0]:
from pyspark import SparkContext
sc = SparkContext.getOrCreate()

## 實例示範

### 範例一

In [8]:
! wget "https://raw.githubusercontent.com/ywchiu/pyspark/master/data/cyber_breaches.tsv"

--2020-05-07 10:58:07--  https://raw.githubusercontent.com/ywchiu/pyspark/master/data/cyber_breaches.tsv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 256462 (250K) [text/plain]
Saving to: ‘cyber_breaches.tsv’

cyber_breaches.tsv  100%[===================>] 250.45K  --.-KB/s    in 0.05s   

2020-05-07 10:58:08 (4.56 MB/s) - ‘cyber_breaches.tsv’ saved [256462/256462]



In [9]:
!  head cyber_breaches.tsv

	Number	Name_of_Covered_Entity	State	Business_Associate_Involved	Individuals_Affected	Date_of_Breach	Type_of_Breach	Location_of_Breached_Information	Date_Posted_or_Updated	Summary	breach_start	breach_end	year
1	0	Brooke Army Medical Center	TX		1000	10/16/2009	Theft	Paper	2014-06-30	A binder containing the protected health information (PHI) of up to 1,272 individuals was stolen from a staff member's vehicle.  The PHI included names, telephone numbers, detailed treatment notes, and possibly social security numbers.  In response to the breach, the covered entity (CE) sanctioned the workforce member and developed a new policy requiring on-call staff members to submit any information created during their shifts to the main office instead of adding it to the binder.  Following OCR's investigation, the CE notified the local media about the breach.	2009-10-16		2009
2	1	Mid America Kidney Stone Association, LLC	MO		1000	9/22/2009	Theft	Network Server	2014-05-30	Five desktop computers containing

In [0]:
rdd = sc.textFile("cyber_breaches.tsv")


In [11]:
rdd

cyber_breaches.tsv MapPartitionsRDD[1] at textFile at NativeMethodAccessorImpl.java:0

In [12]:
rdd.take(5)

['\tNumber\tName_of_Covered_Entity\tState\tBusiness_Associate_Involved\tIndividuals_Affected\tDate_of_Breach\tType_of_Breach\tLocation_of_Breached_Information\tDate_Posted_or_Updated\tSummary\tbreach_start\tbreach_end\tyear',
 "1\t0\tBrooke Army Medical Center\tTX\t\t1000\t10/16/2009\tTheft\tPaper\t2014-06-30\tA binder containing the protected health information (PHI) of up to 1,272 individuals was stolen from a staff member's vehicle.  The PHI included names, telephone numbers, detailed treatment notes, and possibly social security numbers.  In response to the breach, the covered entity (CE) sanctioned the workforce member and developed a new policy requiring on-call staff members to submit any information created during their shifts to the main office instead of adding it to the binder.  Following OCR's investigation, the CE notified the local media about the breach.\t2009-10-16\t\t2009",
 "2\t1\tMid America Kidney Stone Association, LLC\tMO\t\t1000\t9/22/2009\tTheft\tNetwork Server\

In [0]:
header = rdd.first()

In [14]:
header

'\tNumber\tName_of_Covered_Entity\tState\tBusiness_Associate_Involved\tIndividuals_Affected\tDate_of_Breach\tType_of_Breach\tLocation_of_Breached_Information\tDate_Posted_or_Updated\tSummary\tbreach_start\tbreach_end\tyear'

In [0]:
# select * from rdd where line != header
content = rdd.filter(lambda line: line != header)

In [16]:
content.take(3)

["1\t0\tBrooke Army Medical Center\tTX\t\t1000\t10/16/2009\tTheft\tPaper\t2014-06-30\tA binder containing the protected health information (PHI) of up to 1,272 individuals was stolen from a staff member's vehicle.  The PHI included names, telephone numbers, detailed treatment notes, and possibly social security numbers.  In response to the breach, the covered entity (CE) sanctioned the workforce member and developed a new policy requiring on-call staff members to submit any information created during their shifts to the main office instead of adding it to the binder.  Following OCR's investigation, the CE notified the local media about the breach.\t2009-10-16\t\t2009",
 "2\t1\tMid America Kidney Stone Association, LLC\tMO\t\t1000\t9/22/2009\tTheft\tNetwork Server\t2014-05-30\tFive desktop computers containing unencrypted electronic protected health information (e-PHI) were stolen from the covered entity (CE).  Originally, the CE reported that over 500 persons were involved, but subsequ

In [0]:
filtered_content = content.filter(lambda line : len(line.split('\t')) >=10)

In [0]:
summary = filtered_content.map(lambda line: line.split('\t')[10])

In [20]:
summary.take(3)

["A binder containing the protected health information (PHI) of up to 1,272 individuals was stolen from a staff member's vehicle.  The PHI included names, telephone numbers, detailed treatment notes, and possibly social security numbers.  In response to the breach, the covered entity (CE) sanctioned the workforce member and developed a new policy requiring on-call staff members to submit any information created during their shifts to the main office instead of adding it to the binder.  Following OCR's investigation, the CE notified the local media about the breach.",
 "Five desktop computers containing unencrypted electronic protected health information (e-PHI) were stolen from the covered entity (CE).  Originally, the CE reported that over 500 persons were involved, but subsequent investigation showed that about 260 persons were involved.  The ePHI included demographic and financial information. The CE provided breach notification to affected individuals and HHS.  Following the breach

In [23]:
words = summary.flatMap(lambda line : line.lower().split())
words.take(3)

['a', 'binder', 'containing']

In [0]:
wordCounts = words.countByValue()

In [0]:
#wordCounts

### 範例二

In [0]:
splited_data = filtered_content.map(lambda line: line.split('\t'))

In [31]:
splited_data.take(3)

[['1',
  '0',
  'Brooke Army Medical Center',
  'TX',
  '',
  '1000',
  '10/16/2009',
  'Theft',
  'Paper',
  '2014-06-30',
  "A binder containing the protected health information (PHI) of up to 1,272 individuals was stolen from a staff member's vehicle.  The PHI included names, telephone numbers, detailed treatment notes, and possibly social security numbers.  In response to the breach, the covered entity (CE) sanctioned the workforce member and developed a new policy requiring on-call staff members to submit any information created during their shifts to the main office instead of adding it to the binder.  Following OCR's investigation, the CE notified the local media about the breach.",
  '2009-10-16',
  '',
  '2009'],
 ['2',
  '1',
  'Mid America Kidney Stone Association, LLC',
  'MO',
  '',
  '1000',
  '9/22/2009',
  'Theft',
  'Network Server',
  '2014-05-30',
  "Five desktop computers containing unencrypted electronic protected health information (e-PHI) were stolen from the cov

In [0]:
breach_type = splited_data.map(lambda e: ( e[7],int(e[5]) ) )

In [34]:
breach_type.take(5)

[('Theft', 1000),
 ('Theft', 1000),
 ('Theft', 501),
 ('Loss', 3800),
 ('Theft', 5257)]

In [0]:
breach_type_cnt = breach_type.reduceByKey(lambda x,y: x+ y)

In [36]:
breach_type_cnt.take(3)

[('Theft', 16515554), ('Loss', 7254286), ('Hacking/IT Incident', 1878870)]

In [0]:
ret = breach_type_cnt.sortBy(lambda a: -a[1])

In [39]:
ret.take(3)

[('Theft', 16515554), ('Loss', 7254286), ('Unknown', 1918312)]

## 啟用 SparkSQL

In [0]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

In [41]:
! wget "https://raw.githubusercontent.com/ywchiu/pyspark/master/data/cyber_breaches.csv"


--2020-05-07 11:55:46--  https://raw.githubusercontent.com/ywchiu/pyspark/master/data/cyber_breaches.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 275032 (269K) [text/plain]
Saving to: ‘cyber_breaches.csv’

cyber_breaches.csv  100%[===================>] 268.59K  --.-KB/s    in 0.05s   

2020-05-07 11:55:46 (5.26 MB/s) - ‘cyber_breaches.csv’ saved [275032/275032]



In [0]:
df = sqlContext.read.csv('cyber_breaches.csv',  header = True)

In [43]:
df.show(5)

+-------------------+------+----------------------+-----+---------------------------+--------------------+--------------+--------------+--------------------------------+----------------------+--------------------+------------+----------+----+
|                _c0|Number|Name_of_Covered_Entity|State|Business_Associate_Involved|Individuals_Affected|Date_of_Breach|Type_of_Breach|Location_of_Breached_Information|Date_Posted_or_Updated|             Summary|breach_start|breach_end|year|
+-------------------+------+----------------------+-----+---------------------------+--------------------+--------------+--------------+--------------------------------+----------------------+--------------------+------------+----------+----+
|                  1|     0|  Brooke Army Medic...|   TX|                       null|                1000|    10/16/2009|         Theft|                           Paper|            2014-06-30|A binder containi...|  2009-10-16|        NA|2009|
|                  2|     1|

In [44]:
df.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- Number: string (nullable = true)
 |-- Name_of_Covered_Entity: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Business_Associate_Involved: string (nullable = true)
 |-- Individuals_Affected: string (nullable = true)
 |-- Date_of_Breach: string (nullable = true)
 |-- Type_of_Breach: string (nullable = true)
 |-- Location_of_Breached_Information: string (nullable = true)
 |-- Date_Posted_or_Updated: string (nullable = true)
 |-- Summary: string (nullable = true)
 |-- breach_start: string (nullable = true)
 |-- breach_end: string (nullable = true)
 |-- year: string (nullable = true)



In [0]:
# select * from df where state is not null;
from pyspark.sql.functions import col
filtered_df = df.filter(col('State').isNotNull())

In [46]:
filtered_df

DataFrame[_c0: string, Number: string, Name_of_Covered_Entity: string, State: string, Business_Associate_Involved: string, Individuals_Affected: string, Date_of_Breach: string, Type_of_Breach: string, Location_of_Breached_Information: string, Date_Posted_or_Updated: string, Summary: string, breach_start: string, breach_end: string, year: string]

In [47]:
filtered_df.show(5)

+---+------+----------------------+-----+---------------------------+--------------------+--------------+--------------+--------------------------------+----------------------+--------------------+------------+----------+----+
|_c0|Number|Name_of_Covered_Entity|State|Business_Associate_Involved|Individuals_Affected|Date_of_Breach|Type_of_Breach|Location_of_Breached_Information|Date_Posted_or_Updated|             Summary|breach_start|breach_end|year|
+---+------+----------------------+-----+---------------------------+--------------------+--------------+--------------+--------------------------------+----------------------+--------------------+------------+----------+----+
|  1|     0|  Brooke Army Medic...|   TX|                       null|                1000|    10/16/2009|         Theft|                           Paper|            2014-06-30|A binder containi...|  2009-10-16|        NA|2009|
|  2|     1|  Mid America Kidne...|   MO|                       null|                1000|  

In [48]:
filtered_df.select('Type_of_Breach', 'Individuals_Affected').show(3)

+--------------+--------------------+
|Type_of_Breach|Individuals_Affected|
+--------------+--------------------+
|         Theft|                1000|
|         Theft|                1000|
|         Theft|                 501|
+--------------+--------------------+
only showing top 3 rows



In [49]:
filtered_df.select(col('Type_of_Breach'), col('Individuals_Affected')).show(3)

+--------------+--------------------+
|Type_of_Breach|Individuals_Affected|
+--------------+--------------------+
|         Theft|                1000|
|         Theft|                1000|
|         Theft|                 501|
+--------------+--------------------+
only showing top 3 rows



In [50]:
filtered_df.select(col('Type_of_Breach').alias('breach'), col('Individuals_Affected')).show(3)

+------+--------------------+
|breach|Individuals_Affected|
+------+--------------------+
| Theft|                1000|
| Theft|                1000|
| Theft|                 501|
+------+--------------------+
only showing top 3 rows



In [51]:
filtered_df.filter(col('Type_of_Breach') == 'Theft').show(5)


+---+------+----------------------+-----+---------------------------+--------------------+--------------+--------------+--------------------------------+----------------------+--------------------+------------+----------+----+
|_c0|Number|Name_of_Covered_Entity|State|Business_Associate_Involved|Individuals_Affected|Date_of_Breach|Type_of_Breach|Location_of_Breached_Information|Date_Posted_or_Updated|             Summary|breach_start|breach_end|year|
+---+------+----------------------+-----+---------------------------+--------------------+--------------+--------------+--------------------------------+----------------------+--------------------+------------+----------+----+
|  1|     0|  Brooke Army Medic...|   TX|                       null|                1000|    10/16/2009|         Theft|                           Paper|            2014-06-30|A binder containi...|  2009-10-16|        NA|2009|
|  2|     1|  Mid America Kidne...|   MO|                       null|                1000|  

In [52]:
filtered_df.filter(col('Type_of_Breach') == 'Theft').select('Type_of_Breach', 'Individuals_Affected').show(5)


+--------------+--------------------+
|Type_of_Breach|Individuals_Affected|
+--------------+--------------------+
|         Theft|                1000|
|         Theft|                1000|
|         Theft|                 501|
|         Theft|                5257|
|         Theft|                 857|
+--------------+--------------------+
only showing top 5 rows



In [54]:
filtered_df.filter(col('Type_of_Breach') == 'Theft')\
    .select('Type_of_Breach', 'Individuals_Affected')\
    .show(5)


+--------------+--------------------+
|Type_of_Breach|Individuals_Affected|
+--------------+--------------------+
|         Theft|                1000|
|         Theft|                1000|
|         Theft|                 501|
|         Theft|                5257|
|         Theft|                 857|
+--------------+--------------------+
only showing top 5 rows



In [55]:
(filtered_df.filter(col('Type_of_Breach') == 'Theft')
    .select('Type_of_Breach', 'Individuals_Affected')
).show(5)


+--------------+--------------------+
|Type_of_Breach|Individuals_Affected|
+--------------+--------------------+
|         Theft|                1000|
|         Theft|                1000|
|         Theft|                 501|
|         Theft|                5257|
|         Theft|                 857|
+--------------+--------------------+
only showing top 5 rows



In [0]:
# select type_of_breach, sum(indivisual_affected) from df group by type_of_breach;
from pyspark.sql import functions
cnt_df = (filtered_df
    .groupby(col('Type_of_Breach'))
    .agg(functions.sum(col('Individuals_Affected')))
)

In [59]:
cnt_df.show(5)

+--------------------+-------------------------+
|      Type_of_Breach|sum(Individuals_Affected)|
+--------------------+-------------------------+
|       Loss, Unknown|                  13035.0|
|         Loss, Other|                  34534.0|
|  Theft, Loss, Other|                   2600.0|
|Unauthorized Acce...|                   4354.0|
|Theft, Hacking/IT...|                  27800.0|
+--------------------+-------------------------+
only showing top 5 rows



In [0]:
# select type_of_breach, sum(Individuals_Affected) from df group by type_of_breach order by sum(Individuals_Affected) desc;
from pyspark.sql import functions
cnt_df = (filtered_df
    .groupby(col('Type_of_Breach'))
    .agg(functions.sum(col('Individuals_Affected')).alias('affected_numbers'))
    .sort(functions.desc('affected_numbers'))
)

In [62]:
cnt_df.show(5)

+--------------------+----------------+
|      Type_of_Breach|affected_numbers|
+--------------------+----------------+
|               Theft|     1.6515554E7|
|                Loss|       7254286.0|
|             Unknown|       1918312.0|
| Hacking/IT Incident|       1878870.0|
|Unauthorized Acce...|       1424227.0|
+--------------------+----------------+
only showing top 5 rows



## 小任務
- 請幫我找出所有資安影響事件中，哪個州(State)被影響的人數最多